
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>


1. Todas las empresas cuyo nombre sea 'Babelgum'. Recupera solo su campo `name`.
2. Todas las empresas que tienen más de 5000 empleados. Limita la búsqueda a 20 empresas y ordénalas por **número de empleados**.
3. Todas las empresas fundadas entre 2000 y 2005, incluyendo ambos años. Recupera solo los campos `name` y `founded_year`.
4. Todas las empresas que tuvieron una valoración de más de 100.000.000 y que fueron fundadas antes de 2010. Recupera solo los campos `name` e `ipo`.
5. Todas las empresas que tienen menos de 1000 empleados y que fueron fundadas antes de 2005. Ordénalas por el número de empleados y limita la búsqueda a 10 empresas.
6. Todas las empresas que no incluyen el campo `partners`.
7. Todas las empresas que tienen un valor nulo en el campo `category_code`.
8. Todas las empresas que tienen al menos 100 empleados pero menos de 1000. Recupera solo los campos `name` y `number of employees`.
9. Ordena todas las empresas por su precio de IPO en orden descendente.
10. Recupera las 10 empresas con más empleados, ordenadas por el `número de empleados`.
11. Todas las empresas fundadas en el segundo semestre del año. Limita tu búsqueda a 1000 empresas.
12. Todas las empresas fundadas antes del año 2000 que tienen un monto de adquisición superior a 10.000.000.
13. Todas las empresas que han sido adquiridas después de 2010, ordénalas por el monto de adquisición, y recupera solo los campos `name` y `acquisition`.
14. Ordena las empresas por su `founded year`, recuperando solo los campos `name` y `founded year`.
15. Todas las empresas que han sido fundadas en los primeros siete días del mes, incluyendo el séptimo. Ordénalas por su `acquisition price` en orden descendente. Limita la búsqueda a 10 documentos.
16. Todas las empresas en la categoría 'web' que tienen más de 4000 empleados. Ordénalas por la cantidad de empleados en orden ascendente.
17. Todas las empresas cuyo monto de adquisición es superior a 10.000.000, y cuya moneda es 'EUR'.
18. Todas las empresas que han sido adquiridas en el primer trimestre del año. Limita la búsqueda a 10 empresas, y recupera solo los campos `name` y `acquisition`.

In [40]:
import json
import pandas as pd
import numpy as np
from datetime import datetime

import pymongo

pd.set_option("display.max_columns", None)

In [7]:
cliente = pymongo.MongoClient()
print(cliente)

# nos conectamos con una BBDD 
mydb = cliente['LabMongo']

#Veamos además las colecciones de la BBDD
mydb.list_collection_names()
companies = mydb["Companies"]


MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


##### 1. Todas las empresas cuyo nombre sea 'Babelgum'. Recupera solo su campo `name`.


In [10]:
empresas_babelgum = list(companies.find({"name":"Babelgum"}))
print(f"Tenemos {len(empresas_babelgum)} empresas cuyo nombre es Babelgum.")

Tenemos 1 empresas cuyo nombre es Babelgum.


##### 2. Todas las empresas que tienen más de 5000 empleados. Limita la búsqueda a 20 empresas y ordénalas por **número de empleados**.

In [20]:
empresas_5000 = list(companies.find({"number_of_employees":{"$gt":5000}}).limit(20).sort({"number_of_employees":-1}))
print(f"Hemos calculado {len(empresas_5000)} empresas con más de 5000 empleados.")


Hemos calculado 20 empresas con más de 5000 empleados.


3. Todas las empresas fundadas entre 2000 y 2005, incluyendo ambos años. Recupera solo los campos `name` y `founded_year`.

In [23]:
empresas_2000=list(companies.find({'founded_year': {'$in':[2000,2001,2002,2003,2004,2005]}}, {'name':1, 'founded_year':1}))
print(f"Hay {len(empresas_2000)} empresas fundadas entre 2000 y 2005.")


Hay 3734 empresas fundadas entre 2000 y 2005.


4. Todas las empresas que tuvieron una valoración de más de 100.000.000 y que fueron fundadas antes de 2010. Recupera solo los campos `name` e `ipo`.

In [37]:
cond1 = {"ipo.valuation_amount": {'$gt':100000000}}
cond2 = {"founded_year": {'$lt':2010}}
df_4= pd.DataFrame(companies.find({'$and':[cond1, cond2]}, {'_id':0, 'name':1, 'ipo':1}))

df_4 = pd.concat([df_4, df_4["ipo"].apply(pd.Series)], axis=1)
df_4.drop(columns=["ipo"], inplace=True)
df_4.head(2)

,name,valuation_amount,valuation_currency_code,pub_year,pub_month,pub_day,stock_symbol
0,Facebook,104000000000,USD,2012.0,5.0,18.0,NASDAQ:FB
1,Twitter,18100000000,USD,2013.0,11.0,7.0,NYSE:TWTR


5. Todas las empresas que tienen menos de 1000 empleados y que fueron fundadas antes de 2005. Ordénalas por el número de empleados y limita la búsqueda a 10 empresas.

In [42]:
cond5_1 = {"number_of_employees":{'$lt':1000}}
cond5_2 = {'founded_year':{"$lt":2005}}
df_5=pd.DataFrame(companies.find({"$and": [cond5_1, cond5_2]}).sort("number_of_employees").limit(10))
df_5.head(1)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297dbc,Skype,skype,http://www.crunchbase.com/company/skype,http://www.skype.com,http://blogs.skype.com,http://feeds.feedburner.com/shareskypeen,skype,software,0,2003,8.0,NaN,NaN,NaN,NaN,,"phone, voip, sweden, skype",,,94117733,VoIP Software,Tue Jul 03 05:20:31 UTC 2007,Wed Dec 11 13:03:57 UTC 2013,<p>Skype is a software application that allows...,"{'available_sizes': [[[150, 66], 'assets/image...","[{'name': 'Skype', 'permalink': 'skype'}, {'na...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'fring Ltd', 'permali...","[{'title': '', 'is_past': False, 'provider': {...",$76.8M,"[{'id': 79, 'round_code': 'b', 'source_url': '...","[{'funding_round': {'round_code': 'a', 'source...","{'price_amount': 2600000000, 'price_currency_c...","[{'price_amount': 150000000, 'price_currency_c...","[{'description': 'Global HQ', 'address1': '23-...","[{'id': 552, 'description': 'Skype released pu...",None,"[{'embed_code': '<iframe src=""http://blip.tv/p...","[{'available_sizes': [[[150, 130], 'assets/ima...",[{'external_url': 'http://www.ewdn.com/2011/06...,[]


6. Todas las empresas que no incluyen el campo `partners`.

In [44]:
df_6 = pd.DataFrame(companies.find({'partners': {'$exists': False}}))
df_6


""


7. Todas las empresas que tienen un valor nulo en el campo `category_code`.


In [65]:
df7 = pd.DataFrame(companies.find({"category_code":{"$eq":None}}))
df7.head(1)


,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd6752980f6,Collective,collective,http://www.crunchbase.com/company/collective,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,Thu Sep 26 13:15:02 UTC 2013,Thu Sep 26 13:15:02 UTC 2013,None,None,[],[],[],[],$0,[],[],None,[],[],[],None,[],[],[],[]


8. Todas las empresas que tienen al menos 100 empleados pero menos de 1000. Recupera solo los campos `name` y `number of employees`.


In [69]:
cond10_1={"number_of_employees":{"$gte":100}}
cond10_2={"number_of_employees":{"$lt":1000}}

df8 = pd.DataFrame(companies.find({"$and":[cond10_1, cond10_2]}, {"name":1, "number_of_employees":1}))
df8.head(2)

,_id,name,number_of_employees
0,52cdef7c4bab8bd675297d8b,AdventNet,600
1,52cdef7c4bab8bd675297da7,AddThis,120


9. Ordena todas las empresas por su precio de IPO en orden descendente.


In [73]:
df9 = pd.DataFrame(companies.find({"ipo.valuation_amount":{"$ne":None}}).sort({"ipo.valuation_amount":-1}))
df9.head(2)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7e4bab8bd67529a8b4,GREE,gree,http://www.crunchbase.com/company/gree,http://www.gree-corp.com,,,gree_corp,games_video,700.0,2004.0,12.0,7.0,None,None,None,None,"mobile-web, japan, tokyo, social-network, mobi...",None,inquiry@gree-corp.com,,"Internet media business,SNS, free game",Sat Dec 20 16:42:57 UTC 2008,Tue Jan 01 21:37:04 UTC 2013,<p>GREE provides Japan&#8217;s leading mobile ...,"{'available_sizes': [[[150, 61], 'assets/image...",[],"[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Mobage-town', 'perma...",[],¥464M,"[{'id': 18872, 'round_code': 'unattributed', '...","[{'funding_round': {'round_code': 'seed', 'sou...",None,"[{'price_amount': 104000000, 'price_currency_c...","[{'description': 'Headquarters', 'address1': '...","[{'id': 1021, 'description': 'IPO', 'stoned_ye...","{'valuation_amount': 108960000000, 'valuation_...",[],"[{'available_sizes': [[[133, 150], 'assets/ima...",[],[]
1,52cdef7c4bab8bd675297d8e,Facebook,facebook,http://www.crunchbase.com/company/facebook,http://facebook.com,http://blog.facebook.com,http://blog.facebook.com/atom.php,facebook,social,5299.0,2004.0,2.0,1.0,None,None,None,,"facebook, college, students, profiles, network...",,,,Social network,Fri May 25 21:22:15 UTC 2007,Thu Nov 21 19:40:55 UTC 2013,<p>Facebook is the world&#8217;s largest socia...,"{'available_sizes': [[[150, 61], 'assets/image...","[{'name': 'Facebook Platform', 'permalink': 'f...","[{'is_past': False, 'title': 'Founder and CEO,...","[{'competitor': {'name': 'MySpace', 'permalink...","[{'title': '', 'is_past': False, 'provider': {...",$2.43B,"[{'id': 2, 'round_code': 'angel', 'source_url'...","[{'funding_round': {'round_code': 'seed', 'sou...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Headquarters', 'address1': '...","[{'id': 108, 'description': 'Facebook adds com...","{'valuation_amount': 104000000000, 'valuation_...",[],"[{'available_sizes': [[[150, 68], 'assets/imag...",[{'external_url': 'http://latimesblogs.latimes...,[]


10. Recupera las 10 empresas con más empleados, ordenadas por el `número de empleados`.

In [75]:
df10 = pd.DataFrame(companies.find({"number_of_employees":{"$ne":None}}).sort({"number_of_employees":-1}).limit(10))
df10.head(2)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7d4bab8bd67529941a,Siemens,siemens,http://www.crunchbase.com/company/siemens,http://www.siemens.com,,,Siemens,hardware,405000,1847,NaN,NaN,None,None,None,None,"automation, building-technologies, drive-techn...",,contact@siemens.com,49 89 636 34134,Electronics and Electrical Engineering,Thu Jul 31 09:29:43 UTC 2008,Thu Nov 28 20:32:55 UTC 2013,"<p>Siemens AG, an electronics and electrical e...","{'available_sizes': [[[150, 34], 'assets/image...",[],"[{'is_past': False, 'title': 'Chairman, Superv...",[],"[{'title': 'Application Development', 'is_past...",$8.9M,"[{'id': 17062, 'round_code': 'grant', 'source_...",[],None,"[{'price_amount': 418000000, 'price_currency_c...","[{'description': '', 'address1': 'Wittelsbache...","[{'id': 35852, 'description': 'Siemens Receive...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<iframe width=""420"" height=""3...","[{'available_sizes': [[[150, 92], 'assets/imag...",[],[]
1,52cdef7c4bab8bd67529856a,IBM,ibm,http://www.crunchbase.com/company/ibm,http://www.ibm.com,,,IBM,software,388000,1896,NaN,NaN,None,None,None,,,,ews@us.ibm.com,914-499-1900,,Fri Mar 14 22:55:52 UTC 2008,Sat Jan 04 02:56:24 UTC 2014,"<p>IBM, acronym for International Business Mac...","{'available_sizes': [[[150, 60], 'assets/image...","[{'name': 'Lotus Notes', 'permalink': 'lotus-n...","[{'is_past': False, 'title': 'Chairman of the ...","[{'competitor': {'name': 'BEA Systems', 'perma...","[{'title': '', 'is_past': False, 'provider': {...",$0,[],"[{'funding_round': {'round_code': 'c', 'source...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Corporate Headquarters', 'ad...","[{'id': 10471, 'description': 'IBM Completes A...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 93], 'assets/imag...","[{'external_url': 'http://mashpedia.com/IBM', ...",[]


11. Todas las empresas fundadas en el segundo semestre del año. Limita tu búsqueda a 1000 empresas.

In [80]:
cond11_1={"founded_month":{"$gt":6}}
cond11_2={"founded_month":{"$ne":None}}

df11 = pd.DataFrame(companies.find({"$and":[cond11_1, cond11_2]}).limit(1000))
df11.head(2)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,2005,10,17.0,1.0,"wiki, seattle, elowitz, media-industry, media-...",,info@wetpaint.com,206.859.6300,Technology Platform Company,2007-05-25 06:51:27,Sun Dec 08 07:15:44 UTC 2013,<p>Wetpaint is a technology platform company t...,"{'available_sizes': [[[150, 75], 'assets/image...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...","[{'competitor': {'name': 'Wikia', 'permalink':...",[],$39.8M,"[{'id': 888, 'round_code': 'a', 'source_url': ...",[],"{'price_amount': 30000000, 'price_currency_cod...",[],"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,2005,9,15.0,3.0,"zoho, officesuite, spreadsheet, writer, projec...",,info@zohocorp.com,1-888-204-3539,Online Business Apps Suite,Fri May 25 19:30:28 UTC 2007,Wed Oct 30 00:07:05 UTC 2013,"<p>Zoho offers a suite of Business, Collaborat...","{'available_sizes': [[[150, 55], 'assets/image...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...","[{'competitor': {'name': 'Empressr', 'permalin...",[],$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 388, 'description': 'Zoho Reaches 2 Mi...","[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],NaN,NaN,NaN,NaN


12. Todas las empresas fundadas antes del año 2000 que tienen un monto de adquisición superior a 10.000.000.

In [83]:
cond12_1 = {"founded_year":{"$lt":2000}}
cond12_2 = {"acquisition.price_amount":{"$gt":10000000}}

df12 = pd.DataFrame(companies.find({"$and":[cond12_1, cond12_2]}))
df12.head(1)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,1999,6.0,2.0,NaN,NaN,NaN,None,,None,,888.584.3150,None,Fri Jun 08 12:19:51 UTC 2007,Sat Aug 13 18:02:34 UTC 2011,<p>Postini focuses on two main issues: securit...,"{'available_sizes': [[[150, 61], 'assets/image...","[{'name': 'Postini Solutions', 'permalink': 'p...","[{'is_past': False, 'title': 'Board', 'person'...","[{'competitor': {'name': 'RescueTime', 'permal...",[],$0,[],[],"{'price_amount': 625000000, 'price_currency_co...",[],"[{'description': None, 'address1': '959 Skyway...",[],None,[],[],[],[]


13. Todas las empresas que han sido adquiridas después de 2010, ordénalas por el monto de adquisición, y recupera solo los campos `name` y `acquisition`.

In [112]:
df13 = pd.DataFrame(companies.find({"acquisition.acquired_year":{"$gt":2010}}, {"name":1, "acquisition":1}).sort({"acquisition.price_amount":-1}))
df13.head(1)

,_id,name,acquisition
0,52cdef7c4bab8bd675298876,T-Mobile,"{'price_amount': 39000000000, 'price_currency_..."


14. Ordena las empresas por su `founded year`, recuperando solo los campos `name` y `founded year`.

In [94]:
df14 = pd.DataFrame(companies.find({"founded_year": {"$ne":None}},{"name":1, "founded_year":1}).sort({"founded_year":-1}))
df14

,_id,name,founded_year
0,52cdef7d4bab8bd675298ea7,iBazar,2013
1,52cdef7f4bab8bd67529bf69,VistaGen Therapeutics,2013
2,52cdef7e4bab8bd67529a3b8,Clowdy,2013
3,52cdef7c4bab8bd675297fec,Fixya,2013
4,52cdef7c4bab8bd6752982d4,Advaliant,2013
...,...,...,...
13131,52cdef7e4bab8bd67529ac9a,Bachmann Industries,1833
13132,52cdef7f4bab8bd67529c453,DuPont,1802
13133,52cdef7e4bab8bd67529b1bc,US Army,1800
13134,52cdef7d4bab8bd675298b5a,SmallWorlds,1800


15. Todas las empresas que han sido fundadas en los primeros siete días del mes, incluyendo el séptimo. Ordénalas por su `acquisition price` en orden descendente. Limita la búsqueda a 10 documentos.

In [97]:
df15 = pd.DataFrame(companies.find({"founded_day": {"$in":[1,2,3,4,5,6,7]}}).sort({"acquisition.price_amount":-1}).limit(10))
df15.head(1)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7d4bab8bd6752989a1,Netscape,netscape,http://www.crunchbase.com/company/netscape,http://netscape.aol.com,,,,software,NaN,1994,4,4,NaN,NaN,NaN,None,None,,,,,Tue May 06 00:27:28 UTC 2008,Thu Nov 14 00:57:06 UTC 2013,<p>Netscape Communications Corporation offers ...,"{'available_sizes': [[[150, 40], 'assets/image...",[],"[{'is_past': False, 'title': 'Chief Scientist'...",[],[],$0,"[{'id': 20468, 'round_code': 'c', 'source_url'...",[],"{'price_amount': 4200000000, 'price_currency_c...",[],"[{'description': '', 'address1': '466 Ellis St...",[],None,[],[],[],[]


16. Todas las empresas en la categoría 'web' que tienen más de 4000 empleados. Ordénalas por la cantidad de empleados en orden ascendente.

In [111]:
cond16_1 = {"category_code":{"$eq":"web"}}
cond16_2 = {"number_of_employees":{"$gt":4000}}
df16 = pd.DataFrame(companies.find({"$and":[cond16_1, cond16_2]}).sort({"number_of_employees":1}))
df16.head(1)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd67529822a,Expedia,expedia,http://www.crunchbase.com/company/expedia,http://www.expedia.com,,,Expedia,web,4400,1996,NaN,NaN,None,None,None,None,"travel, flights, hotels, tickets, airline, vac...",,,1-800-EXPEDIA,,Sun Feb 10 03:29:12 UTC 2008,Sun Dec 08 07:03:33 UTC 2013,<p>Expedia.com is an Internet-based travel age...,"{'available_sizes': [[[150, 115], 'assets/imag...",[],"[{'is_past': False, 'title': 'Director and Vic...","[{'competitor': {'name': 'Farecast', 'permalin...","[{'title': '', 'is_past': False, 'provider': {...",$0,[],"[{'funding_round': {'round_code': 'c', 'source...","{'price_amount': None, 'price_currency_code': ...","[{'price_amount': None, 'price_currency_code':...","[{'description': 'Corporate Office', 'address1...","[{'id': 11691, 'description': 'US AIRWAYS AND ...","{'valuation_amount': None, 'valuation_currency...",[],"[{'available_sizes': [[[150, 93], 'assets/imag...",[{'external_url': 'http://www.urlaubs-rabatte....,[]


17. Todas las empresas cuyo monto de adquisición es superior a 10.000.000, y cuya moneda es 'EUR'.

In [115]:
cond17_1 = {"acquisition.price_currency_code":{"$eq":"EUR"}}
cond17_2 = {"acquisition.price_amount":{"$gt":10000000}}

df17 = pd.DataFrame(companies.find({"$and":[cond17_1, cond17_2]}))
df17.head(1)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297f02,ZYB,zyb,http://www.crunchbase.com/company/zyb,http://zyb.com,http://blog.zyb.com/,http://blog.zyb.com/index.php/feed/,None,mobile,25.0,NaN,NaN,NaN,None,None,None,None,"mobile, social-networking, socialnetwork",,contact@zyb.com,,Mobile Social Networking,Wed Aug 22 05:14:51 UTC 2007,Tue Jul 02 03:00:50 UTC 2013,<p>ZYB is a mobile phone utility and social ne...,"{'available_sizes': [[[32, 32], 'assets/images...","[{'name': 'ZYB | Phone', 'permalink': 'zyb-pho...","[{'is_past': None, 'title': 'CEO and Co-Founde...","[{'competitor': {'name': 'Twitter', 'permalink...","[{'title': '', 'is_past': True, 'provider': {'...",€3M,"[{'id': 389, 'round_code': 'a', 'source_url': ...",[],"{'price_amount': 31500000, 'price_currency_cod...","[{'price_amount': None, 'price_currency_code':...","[{'description': None, 'address1': 'Esromgade ...",[],None,[],[],[],[]


18. Todas las empresas que han sido adquiridas en el primer trimestre del año. Limita la búsqueda a 10 empresas, y recupera solo los campos `name` y `acquisition`.

In [119]:
df18 = pd.DataFrame(companies.find({"acquisition.acquired_month":{"$lt":4}}, {"name":1, "acquisition":1}).limit(10))
df18.head(1)

,_id,name,acquisition
0,52cdef7c4bab8bd675297dab,Kyte,"{'price_amount': None, 'price_currency_code': ..."
